In [35]:
# 'T5 | Text-to-Text Transformer|Text-to-Text Transfer Transformer'
# 'mT5'
    # successor of T5
# 'PanGu-α'
    # GPT-3 as background
# CPM-2
    # mention mT5 as comparison
# T0
openSourceList = ['T5', 'mT5', 'PanGu-α', 'CPM-2','T0','GPT-NeoX-20B','CodeGen','Tk-Instruct','UL2','OPT','YaLM','NLLB','BLOOM','GLM','Flan-T5','mT0','Galatica','BLOOMZ','OPT-IML','Pythia','LLaMA','Vicuna','ChatGLM','CodeGeeX','Koala']
closeSourceList = ['GShard','GPT-3','GPT 3','LaMDA','HyperCLOVA','Codex','ERNIE 3','Jurassic-1','FLAN','MT-NLG','Yuan 1.0','Anthropic','WebGPT','Gopher','ERNIE 3.0 Titan','GLaM','InstructGPT','ChatGPT','AlphaCode','Chinchilla','PaLM','Cohere','AlexaTM','Luminous','Sparrow','WeLM','U-PaLM','Flan-PaLM','Flan-U-PaLM','Alpaca','GPT-4' ,'GPT 4','PanGU-Σ','Claude']

# we can not find 'PanGU-Σ', 'Luminous' in close Source index, cannot find 'Galatica', 'YaLM', 'PanGu-α' in open source list , so delete them.
openSourceList = ['T5', 'mT5', 'CPM-2','T0','GPT-NeoX-20B','CodeGen','Tk-Instruct','UL2','OPT','NLLB','BLOOM','GLM','Flan-T5','mT0','BLOOMZ','OPT-IML','Pythia','LLaMA','Vicuna','ChatGLM','CodeGeeX','Koala']
closeSourceList = ['GShard','GPT-3','GPT 3','LaMDA','HyperCLOVA','Codex','ERNIE 3','Jurassic-1','FLAN','MT-NLG','Yuan 1.0','Anthropic','WebGPT','Gopher','ERNIE 3.0 Titan','GLaM','InstructGPT','ChatGPT','AlphaCode','Chinchilla','PaLM','Cohere','AlexaTM','Sparrow','WeLM','U-PaLM','Flan-PaLM','Flan-U-PaLM','Alpaca','GPT-4' ,'GPT 4','Claude']
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
readfile = "/data/jx4237data/DataForChatGPTinnovationWaves/"
df = pd.read_csv(readfile + 'LLMs0819.csv',dtype=object)

In [4]:
# some paper we need to delete:
# “fake” palm paper 1909.02134
# fake openai 1506.04006
delete_list = ['1506.04006','1909.02134']

In [5]:
df = df[~df['id'].isin(delete_list)]

In [6]:
# model2paper
model2paper = {}

In [ ]:
model = 'PanGu'
df[df['abstract'].str.contains("\\b%s\\b"%model)  | df['title'].str.contains("\\b%s\\b"%model)]

In [ ]:
# 得到model2paper
from tqdm import tqdm
for model in tqdm(openSourceList+closeSourceList):
    if model in model2paper:
        print('error')
    model2paper[model] = set()
    for paper in df[df['abstract'].str.contains("\\b%s\\b"%model)  | df['title'].str.contains("\\b%s\\b"%model)]['id']:
        model2paper[model].add(paper)

In [ ]:
# 展示下数量
for k,v in model2paper.items():
    print(k+','+str(len(v)))

In [ ]:
# openset, close set; open - close是纯open，close - open是纯close.
openPaperSet = set()
closePaperSet = set()
for k,v in model2paper.items():
    if k in closeSourceList:
        for paper in v:
            closePaperSet.add(paper)
    if k in openSourceList:
        for paper in v:
            openPaperSet.add(paper)
print('open & close',len(closePaperSet & openPaperSet))
print('close',len(closePaperSet - openPaperSet))
print('open',len(openPaperSet - closePaperSet))

In [11]:
# # 假如提到ChatGPT且只提到的不算（这样少了大约600篇文章）
# # openset, close set; open - close是纯open，close - open是纯close.
# openPaperSet = set()
# closePaperSet = set()
# for k,v in model2paper.items():
#     if k =='ChatGPT':
#         continue
#     if k in closeSourceList:
#         for paper in v:
#             closePaperSet.add(paper)
#     if k in openSourceList:
#         for paper in v:
#             openPaperSet.add(paper)
# print('open & close',len(closePaperSet & openPaperSet))
# print('close',len(closePaperSet - openPaperSet))
# print('open',len(openPaperSet - closePaperSet))

In [12]:
# paper2model
paper2model = {}
for model, paperSet in model2paper.items():
    for paper in paperSet:
        if paper not in paper2model:
            paper2model[paper] =[model]
        else:
            paper2model[paper].append(model)

In [ ]:
print('open & close',len(closePaperSet & openPaperSet))
print('close',len(closePaperSet - openPaperSet))
print('open',len(openPaperSet - closePaperSet))
print('all',len(openPaperSet | closePaperSet))
print('mention close',len(closePaperSet))
print('mention open',len(openPaperSet))

In [14]:
pureOpen = openPaperSet - closePaperSet
pureClose = closePaperSet - openPaperSet
mixed = openPaperSet & closePaperSet
all = openPaperSet | closePaperSet

In [15]:
paper2pubdate = {}
for i in range(len(df)):
    paper2pubdate[list(df['id'])[i]] = list(df['update_date'])[i]

In [ ]:
from collections import Counter
# 记录pureOpen每个月份，提及top model的数量
# 月份，top model, count
pureOpenModelTrend = {}
for paper in all:
    if paper2pubdate[paper] not in pureOpenModelTrend:
        tempCounter = Counter()
        for modelname in paper2model[paper]:
            tempCounter[modelname] += 1
        pureOpenModelTrend[paper2pubdate[paper]] = tempCounter
    else:
        for modelname in paper2model[paper]:
            pureOpenModelTrend[paper2pubdate[paper]][modelname] += 1

# full time
temp = Counter()
for k,v in pureOpenModelTrend.items():
    temp = temp + v
print('all    ',sorted(temp.items(),key=lambda x : -x[1]))

for k,v in sorted(pureOpenModelTrend.items(),key=lambda x : x[0]):
    print(k,sorted(v.items(),key=lambda x : -x[1]))


In [ ]:
from tqdm import tqdm
from dateutil import parser
flag = 0
for temp_model in tqdm(temp.keys()):
    temp_dict = {}
    for k,v in pureOpenModelTrend.items():
        temp_dict[parser.parse(k)] = v[temp_model]
    sf = pd.Series(temp_dict)
    sf = sf.sort_index()
    # if temp_model in openSourceList:
    #     temp_model = temp_model+' (Public Available)'
    # else:
    #     temp_model = temp_model+' (Closed Source)'
    if flag == 0:
        df_cumu = pd.DataFrame({temp_model:sf.values},index = sf.index)
        flag = 1
    else:
        df_cumu[temp_model] = sf

In [18]:
df_for_rolling = pd.DataFrame()
df_for_sum = pd.DataFrame()

df_for_sum['closed model'] = df_cumu[closeSourceList].sum(axis=1)
df_for_sum['open model'] = df_cumu[openSourceList].sum(axis=1)

for model_name in df_cumu.columns:
    df_for_rolling['%s ( 10 days rolling average )'%model_name] = df_cumu[model_name].rolling('10D').mean()
df_for_rolling['Open Sourced model ( 10 days rolling average )'] = df_for_sum['open model'].rolling('10D').mean()
df_for_rolling['Closed Sourced Model ( 10 days rolling average )'] = df_for_sum['closed model'].rolling('10D').mean()

df_cumu['date'] = df_cumu.index
df_for_rolling['date'] = df_cumu.index
df_for_sum['date'] = df_cumu.index

In [ ]:
# the index is time, we can do many time-series operation
df_cumu['2021-1':]

In [24]:
# top 15 mentioned models
top_mentioned = list(df_for_rolling.sum(numeric_only=True, axis=0).sort_values(ascending=False).index[:10])

In [51]:
import plotly.express as px
df = df_for_rolling[['Open Sourced model ( 10 days rolling average )','Closed Sourced Model ( 10 days rolling average )']]
df['date'] = df.index
fig = px.line(df, x='date', y=df.columns,
              hover_data={'date': "|%B %d, %Y"},
              title='Open sourced v.s Closed sourced: ChatGPT is the gamechanger?',)
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")

fig.add_vrect(x0="2022-11-25", x1="2022-12-06",
              annotation_text="2020-11-30: ChatGPT launched ! ", annotation_position="top right",
              fillcolor="green", opacity=0.5, line_width=1, )

fig.show()

In [162]:
df = df_for_sum[['closed model', 'open model']].cumsum()
df['date'] = df.index
fig = px.line(df, x='date', y=df.columns,
              hover_data={'date': "|%B %d, %Y"},
              title='Open sourced v.s Closed sourced: Before ChatGPT, GPT-3 drew more attention than t-5. (Cumulative Distribution)',
              log_y=True,
              )
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")
fig.add_vrect(x0="2022-11-25", x1="2022-12-06",
              annotation_text="2020-11-30: ChatGPT", annotation_position="top right",
              fillcolor="green", opacity=0.25, line_width=1,)



fig.add_vrect(x0="2019-11-21", x1="2019-12-02",
              annotation_text="2019-10-23: T5 model", annotation_position="top left",
              fillcolor="grey", opacity=0.25, line_width=1)


fig.add_vrect(x0="2020-06-06", x1="2020-06-20",
              annotation_text="2020-06-11: GPT-3 model", annotation_position="top left",
              fillcolor="blue", opacity=0.25, line_width=1)


fig.show()

In [160]:
top_mentioned = list(df_for_rolling.sum(numeric_only=True, axis=0).sort_values(ascending=False).index[:10])
df = df_for_rolling[top_mentioned]
df['date'] = df.index
fig = px.line(df, x='date', y=df.columns,
              hover_data={'date': "|%B %d, %Y"},
              title='Open source: T5 (Google), OPT (Meta), and LLaMA (Meta) \n\n Closed Source: GPTs (GPT-3, CodeX, ChatGPT,GPT-4) plus PaLM (Google)',
)
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")
fig.show()

In [161]:
top_mentioned = list(df_cumu.sum(numeric_only=True, axis=0).sort_values(ascending=False).index[:10])
df = df_cumu[top_mentioned].cumsum()
df['date'] = df.index
fig = px.line(df, x='date', y=df.columns,
              hover_data={'date': "|%B %d, %Y"},
              title='Open: T5 (Google), OPT (Meta), BLOOM (HuggingFace), and LLaMA (Meta);    Closed: GPTs (GPT-3, CodeX, InsturctGPT, ChatGPT,GPT-4) plus PaLM (Google)',
              log_y= True
)
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")
fig.show()

#### Let's consider the diffusion speed of these models. Yes, do an alignment of the above plots.

X-axes are the days after the model was proposed.

In [140]:
# the first time when a model is proposed.
modelLaunchTime = {}

In [141]:
for paper, modellist in paper2model.items():
    for model in modellist:
        temp_time = parser.parse(paper2pubdate[paper])
        if model not in modelLaunchTime:
            modelLaunchTime[model] = temp_time
        else:
            # if temp_time is earlier, temp_time is potential the launch time.
            if modelLaunchTime[model] > temp_time:
                modelLaunchTime[model] = temp_time


In [ ]:
# record each datedelta(days), how many papers mentioned regarding model name
# timedelta, modelname, count
ModelTrend = {}
for paper in all:
    for modelname in paper2model[paper]:
        timedelta = parser.parse(paper2pubdate[paper]) - modelLaunchTime[modelname]
        if timedelta not in ModelTrend:
            tempCounter = Counter()
            tempCounter[modelname] += 1
            ModelTrend[timedelta] = tempCounter
        else:
            ModelTrend[timedelta][modelname] += 1

# full time
temp = Counter()
for k,v in ModelTrend.items():
    temp = temp + v
print('all    ',sorted(temp.items(),key=lambda x : -x[1]))

for k,v in sorted(ModelTrend.items(),key=lambda x : x[0]):
    print(k,sorted(v.items(),key=lambda x : -x[1]))


In [ ]:
flag = 0
for temp_model in tqdm(temp.keys()):
    temp_dict = {}
    for k,v in ModelTrend.items():
        temp_dict[k] = v[temp_model]
    sf = pd.Series(temp_dict)
    sf = sf.sort_index()
    if flag == 0:
        df_delta = pd.DataFrame({temp_model:sf.values},index = sf.index)
        flag = 1
    else:
        df_delta[temp_model] = sf

In [144]:
df_delta_for_rolling = pd.DataFrame()
df_delta_for_sum = pd.DataFrame()


for model_name in df_delta.columns:
    df_delta_for_rolling['%s ( 10 days rolling average )'%model_name] = df_delta[model_name].rolling('10D').mean()

df_delta['date'] = df_delta.index.days
df_delta_for_rolling['date'] = df_delta.index.days
df_delta_for_sum['date'] = df_delta.index.days

In [165]:
top_mentioned = list(df_delta_for_rolling.sum(numeric_only=True, axis=0).sort_values(ascending=False).index[:10])
df = df_delta_for_rolling[top_mentioned]
df['days'] = df.index.days
fig = px.line(df, x='days', y=df.columns,
              hover_data={'date': "|%B %d, %Y"},
              title='Explosive Growth: GPT-4, ChatGPT, LLaMA;   Founders: GPT-3 and T5',
)
fig.show()

In [166]:
top_mentioned = list(df_delta.sum(numeric_only=True, axis=0).sort_values(ascending=False).index[:10])
df = df_delta[top_mentioned].cumsum()
df['days'] = df.index.days
fig = px.line(df, x='days', y=df.columns,
              hover_data={'date': "|%B %d, %Y"},
              title='Explosive Growth: GPT-4, ChatGPT, LLaMA;   Founders: GPT-3 and T5',
              log_y= True
)
fig.show()